# 🛍️ Retail Sales Analysis (Python EDA)

**Goal:** Analyze retail performance across time, products, and customers. Identify top categories, seasonal patterns, profitable segments, and targetable customers based on RFM.

**Key Questions**
- Which categories/subcategories drive the most **sales** and **profit**?
- How do **regions** perform?
- What is our **seasonality** and **monthly trend**?
- Which customers are **VIPs**, **Loyal**, **At Risk**, or **Regular** (via **RFM**)?
- Are **discounts** helping or hurting **profit**?


## 1. Load & Inspect Data

In [ ]:
import pandas as pd

orders = pd.read_csv("retail_orders.csv", parse_dates=["order_date", "ship_date"])
customers = pd.read_csv("retail_customers.csv", parse_dates=["join_date"])
products = pd.read_csv("retail_products.csv")

orders.shape, customers.shape, products.shape

In [ ]:
orders.head()

In [ ]:
customers.head()

In [ ]:
products.head()

In [ ]:
orders.isnull().sum(), customers.isnull().sum(), products.isnull().sum()

In [ ]:
orders.dtypes

## 2. Basic EDA

In [ ]:
import plotly.express as px

cat_sales = (orders.groupby("category", as_index=False)
             .agg(sales=("sales","sum"), profit=("profit","sum"), quantity=("quantity","sum"))
             .sort_values("sales", ascending=False))
px.bar(cat_sales, x="category", y="sales", title="Sales by Category").show()
px.bar(cat_sales, x="category", y="profit", title="Profit by Category").show()

orders["order_month"] = orders["order_date"].dt.to_period("M").dt.to_timestamp()
monthly = orders.groupby("order_month", as_index=False).agg(sales=("sales","sum"), profit=("profit","sum"))
px.line(monthly, x="order_month", y="sales", title="Monthly Sales Trend").show()
px.line(monthly, x="order_month", y="profit", title="Monthly Profit Trend").show()

region = (orders.groupby("region", as_index=False)
          .agg(sales=("sales","sum"), profit=("profit","sum")))
px.bar(region, x="region", y="sales", title="Sales by Region").show()
px.bar(region, x="region", y="profit", title="Profit by Region").show()

px.scatter(orders, x="discount", y="profit", size="sales", color="category",
           title="Discount vs Profit (bubble size = sales)").show()

## 3. Customer RFM Segmentation

In [ ]:
import numpy as np

snapshot_date = orders["order_date"].max() + pd.Timedelta(days=1)
rfm = (orders.groupby("customer_id")
       .agg(recency=("order_date", lambda x: (snapshot_date - x.max()).days),
            frequency=("order_id", "nunique"),
            monetary=("sales", "sum"))
       .reset_index())

rfm["R_Score"] = pd.qcut(-rfm["recency"], 5, labels=[1,2,3,4,5])
rfm["F_Score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["M_Score"] = pd.qcut(rfm["monetary"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["RFM_Score"] = rfm["R_Score"].astype(int)*100 + rfm["F_Score"].astype(int)*10 + rfm["M_Score"].astype(int)

def rfm_segment(row):
    r,f,m = row["R_Score"], row["F_Score"], row["M_Score"]
    if r>=4 and f>=4 and m>=4: return "VIP"
    if r>=4 and f>=3: return "Loyal"
    if r<=2 and f<=2: return "At Risk"
    if r>=3 and m>=4: return "Big Spenders"
    return "Regular"

rfm["Segment"] = rfm.apply(rfm_segment, axis=1)
rfm.head()

In [ ]:
rfm['Segment'].value_counts()

## 4. Cohort Analysis (Retention)

In [ ]:
orders["cohort"] = orders.groupby("customer_id")["order_date"].transform("min").dt.to_period("M").dt.to_timestamp()
orders["order_month"] = orders["order_date"].dt.to_period("M").dt.to_timestamp()

cohort_pivot = (orders.groupby(["cohort","order_month"])
                .agg(customers=("customer_id","nunique"))
                .reset_index())

cohort_pivot["period_number"] = (
    (cohort_pivot["order_month"].dt.year - cohort_pivot["cohort"].dt.year) * 12 +
    (cohort_pivot["order_month"].dt.month - cohort_pivot["cohort"].dt.month)
)

cohort_size = cohort_pivot[cohort_pivot["period_number"]==0][["cohort","customers"]].rename(columns={"customers":"cohort_size"})
retention = cohort_pivot.merge(cohort_size, on="cohort")
retention["retention"] = (retention["customers"] / retention["cohort_size"]).round(3)

ret_table = retention.pivot_table(index="cohort", columns="period_number", values="retention", fill_value=0)
import plotly.express as px
px.imshow(ret_table, text_auto=True, aspect="auto", title="Cohort Retention (Month 0..)").show()

## 5. Insights & Recommendations

### Insights
- **Category Performance:** Laptops lead sales and profit; Accessories provide steady margins.
- **Regional Performance:** West is strongest; investigate profit gaps in weaker regions.
- **Customer Segmentation:** Big Spenders contribute highest value; Regulars are largest by count.
- **Retention:** Drop is gradual; customers stay engaged for multiple months.

### Recommendations
- **Retain Big Spenders:** VIP benefits, early access, targeted offers.
- **Upsell Regulars:** Promote high-value laptops and bundles.
- **Win-back At Risk:** Re‑activation emails for customers inactive 3+ months.
- **Optimize Discounts:** Test lower discounts where profit is harmed.
